flooded with lots of popups due to fluctuating path - failed (each line at the bottom of this file was a popup)

In [48]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import grey_dilation
import time


In [49]:
def detect_circles_and_obstacles(image):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Green (start)
    green_mask = cv2.inRange(hsv, (40, 40, 40), (80, 255, 255))
    # Blue (end)
    blue_mask = cv2.inRange(hsv, (100, 150, 0), (140, 255, 255))
    # Orange (obstacles)
    orange_mask = cv2.inRange(hsv, (5, 150, 150), (20, 255, 255))

    def find_centroid(mask):
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if contours:
            largest = max(contours, key=cv2.contourArea)
            M = cv2.moments(largest)
            if M["m00"] != 0:
                cx = int(M["m10"] / M["m00"])
                cy = int(M["m01"] / M["m00"])
                return (cx, cy)
        return None

    start = find_centroid(green_mask)
    end = find_centroid(blue_mask)

    return start, end, orange_mask


In [50]:
CELL_SIZE = 20

def pixel_to_grid(pixel, cell_size=CELL_SIZE):
    return (pixel[1] // cell_size, pixel[0] // cell_size)

def grid_to_pixel(grid, cell_size=CELL_SIZE):
    return (grid[1] * cell_size + cell_size // 2, grid[0] * cell_size + cell_size // 2)

def resize_for_pathfinding(mask, factor):
    small = cv2.resize(mask, (mask.shape[1] // factor, mask.shape[0] // factor), interpolation=cv2.INTER_NEAREST)
    return (small > 0).astype(np.uint8)


In [51]:
def inflate_obstacles(grid, inflation_radius=1):
    return grey_dilation(grid, size=(2*inflation_radius+1, 2*inflation_radius+1))


In [52]:
def heuristic(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

def astar(grid, start, end):
    import heapq
    neighbors = [(0,1),(1,0),(-1,0),(0,-1)]
    close_set = set()
    came_from = {}
    gscore = {start:0}
    fscore = {start:heuristic(start, end)}
    oheap = [(fscore[start], start)]

    while oheap:
        _, current = heapq.heappop(oheap)

        if current == end:
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            path.append(start)
            path.reverse()
            return path

        close_set.add(current)
        for i, j in neighbors:
            neighbor = (current[0]+i, current[1]+j)
            tentative_g = gscore[current] + 1
            if 0 <= neighbor[0] < grid.shape[0] and 0 <= neighbor[1] < grid.shape[1]:
                if grid[neighbor[0]][neighbor[1]] == 1:
                    continue
            else:
                continue
            if neighbor in close_set and tentative_g >= gscore.get(neighbor, 0):
                continue
            if tentative_g < gscore.get(neighbor, float('inf')):
                came_from[neighbor] = current
                gscore[neighbor] = tentative_g
                fscore[neighbor] = tentative_g + heuristic(neighbor, end)
                heapq.heappush(oheap, (fscore[neighbor], neighbor))
    return []


In [53]:
def get_direction_changes(path):
    direction_changes = []
    prev_dir = None
    for i in range(1, len(path)):
        dx = path[i][1] - path[i-1][1]
        dy = path[i][0] - path[i-1][0]
        direction = (dx, dy)
        if direction != prev_dir:
            direction_changes.append(grid_to_pixel(path[i-1]))
            prev_dir = direction
    return direction_changes


In [54]:

cap = cv2.VideoCapture(1)  # Change index if needed

last_direction_changes = None  # Store last unique direction changes

while True:
    ret, frame = cap.read()
    if not ret:
        print("Camera error.")
        break

    start_pixel, end_pixel, orange_mask = detect_circles_and_obstacles(frame)
    if not start_pixel or not end_pixel:
        continue

    start_grid = pixel_to_grid(start_pixel)
    end_grid = pixel_to_grid(end_pixel)
    obstacle_grid = resize_for_pathfinding(orange_mask, CELL_SIZE)
    inflated = inflate_obstacles(obstacle_grid, inflation_radius=1)

    path = astar(inflated, start_grid, end_grid)
    if not path:
        continue  # Skip if no path

    # --- Convert direction_changes to tuple of tuples for stable comparison ---
    raw_changes = get_direction_changes(path)
    direction_changes = tuple(tuple(map(int, pt)) for pt in raw_changes)

    if direction_changes != last_direction_changes:
        print("Direction Change Coordinates:", direction_changes)
        last_direction_changes = direction_changes

    # Draw path on copy
    path_image = frame.copy()
    for pt in path:
        px = grid_to_pixel(pt)
        cv2.circle(path_image, px, 2, (0, 0, 0), -1)

    for pt in direction_changes:
        cv2.circle(path_image, pt, 5, (255, 0, 255), -1)

    # Show images
    both = np.hstack((frame, path_image))
    cv2.imshow("Live Pathfinding - Press Q to quit", both)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Direction Change Coordinates: ((550, 430), (550, 310), (230, 310), (230, 210))
Direction Change Coordinates: ((530, 450), (530, 290), (230, 290), (230, 210))
Direction Change Coordinates: ((550, 430), (550, 290), (230, 290), (230, 230))
Direction Change Coordinates: ((510, 390), (510, 290))
Direction Change Coordinates: ((530, 450), (530, 330))
Direction Change Coordinates: ((510, 450), (510, 330))
Direction Change Coordinates: ((530, 450), (530, 330))
Direction Change Coordinates: ((510, 450), (510, 330))
Direction Change Coordinates: ((550, 430), (550, 310))
Direction Change Coordinates: ((510, 450), (510, 310))
Direction Change Coordinates: ((550, 430), (550, 310))
Direction Change Coordinates: ((530, 390), (530, 350))
Direction Change Coordinates: ((530, 450), (530, 350))
Direction Change Coordinates: ((530, 410), (530, 350))
Direction Change Coordinates: ((530, 390), (530, 350))
Direction Change Coordinates: ((530, 450),)
Direction Change Coordinates: ((510, 390), (510, 230), (410